In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
import time
import string
from tqdm import tqdm

In [2]:
class keras_evaluation:
    def __init__(self,data_path):
        self.file_path = [data_path+'' + x+'/'+y for x in os.listdir(data_path) for y in os.listdir(data_path+'/'+x)]

    def evaluate(self,model_path):
        self.model = tf.keras.models.load_model(model_path)
        self.model.compile(optimizer='Adam',loss='mse')

        pred_list = []
        pred_time_list = []
        for i in tqdm(self.file_path):
            label = i.split('/')[1].split('.')[0]
            pred,time_to_predict = self.predict(i)
            pred_time_list.append(time_to_predict)
            pred_list.append(1) if pred == label else pred_list.append(0)

        accuracy = sum(pred_list)/len(pred_list)
        avg_pred_time = sum(pred_time_list)/len(pred_time_list)
        print("Accuracy of the model on {} images is {}".format(len(self.file_path), accuracy))
        print("Average prediction time per image was {} ".format(avg_pred_time))

        return accuracy,avg_pred_time


    def predict(self,image_path):
        tik = time.time()
        img = image.load_img(image_path,target_size=(224,224))
        preprocessed_image = (np.expand_dims(preprocess_input(image.img_to_array(img)),0))#/255

        predictions = self.model.predict(preprocessed_image,verbose=False)#,batch_size=1,verbose=False)
        tok = time.time() - tik

        pred = decode_predictions(predictions)[0][0][0]

        return pred,tok


In [3]:
class tflite_evaluation:
    def __init__(self,data_path):
        self.file_path = [data_path+'' + x+'/'+y for x in os.listdir(data_path) for y in os.listdir(data_path+'/'+x)]

    def evaluate(self,model):
        self.model = model
        self.load_model()
        pred_list = []
        pred_time_list = []
        for i in tqdm(self.file_path):
            label = i.split('/')[1].split('.')[0]
            pred,time_to_predict = self.predict(i)
            pred_time_list.append(time_to_predict)
            pred_list.append(1) if pred == label else pred_list.append(0)

        accuracy = sum(pred_list)/len(pred_list)
        avg_pred_time = sum(pred_time_list)/len(pred_time_list)
        print("Accuracy of the model on {} images is {}".format(len(self.file_path), accuracy))
        print("Average prediction time per image was {} ".format(avg_pred_time))

        return accuracy,avg_pred_time


    def predict(self,image_path):
        tik = time.time()
        img = (image.load_img(image_path,target_size=(224,224)))
        preprocessed_image = preprocess_input(image.img_to_array(img))

        test_image = np.expand_dims(preprocessed_image, axis=0).astype(np.float32)

        input_index = self.interpreter.get_input_details()[0]["index"]
        output_index = self.interpreter.get_output_details()[0]["index"]

        self.interpreter.set_tensor(input_index, test_image)
        self.interpreter.invoke()
        predictions = self.interpreter.get_tensor(output_index)
        tok = time.time() - tik

        pred = decode_predictions(predictions)[0][0][0]

        return pred,tok
    
    def load_model(self):
        self.interpreter = tf.lite.Interpreter(model_path=self.model)
        self.interpreter.allocate_tensors()

In [4]:
compressed_evaluation = tflite_evaluation('val/')
og_evaluation = keras_evaluation('val/')

Evaluating Original Pre-trained Model

In [5]:
og_evaluation.evaluate('./Models/mobilenet_v2.h5')

100%|██████████| 3925/3925 [02:35<00:00, 25.31it/s]

Accuracy of the model on 3925 images is 0.06598726114649682
Average prediction time per image was 0.03925239781665195 


(0.06598726114649682, 0.03925239781665195)

In [6]:
og_evaluation.evaluate('./Models/resnet_50.h5')

100%|██████████| 3925/3925 [05:10<00:00, 12.63it/s]

Accuracy of the model on 3925 images is 0.8300636942675159
Average prediction time per image was 0.07886544768218022 


(0.8300636942675159, 0.07886544768218022)

In [7]:
og_evaluation.evaluate('./Models/vgg16.h5')

100%|██████████| 3925/3925 [07:28<00:00,  8.75it/s]

Accuracy of the model on 3925 images is 0.7579617834394905
Average prediction time per image was 0.11372578444754242 


(0.7579617834394905, 0.11372578444754242)

Evaluating Dynamic Range Quantized Model

In [8]:
compressed_evaluation.evaluate('./Models/mobilenet_v2_dynamic_quant.tflite')

100%|██████████| 3925/3925 [00:36<00:00, 107.55it/s]

Accuracy of the model on 3925 images is 0.062420382165605096
Average prediction time per image was 0.00885416832699138 


(0.062420382165605096, 0.00885416832699138)

In [9]:
compressed_evaluation.evaluate('./Models/resnet_50_dynamic_quant.tflite')

100%|██████████| 3925/3925 [02:11<00:00, 29.92it/s]

Accuracy of the model on 3925 images is 0.8310828025477707
Average prediction time per image was 0.033207292131557584 


(0.8310828025477707, 0.033207292131557584)

In [10]:
compressed_evaluation.evaluate('Models/vgg16_dynamic_quant.tflite')

100%|██████████| 3925/3925 [06:55<00:00,  9.45it/s]

Accuracy of the model on 3925 images is 0.7597452229299363
Average prediction time per image was 0.10519638359167013 


(0.7597452229299363, 0.10519638359167013)

Evaluating Float16 Quantized Model

In [11]:
compressed_evaluation.evaluate('./Models/mobilenet_v2_f16_quant.tflite')

100%|██████████| 3925/3925 [00:53<00:00, 72.90it/s]

Accuracy of the model on 3925 images is 0.06598726114649682
Average prediction time per image was 0.013261378828886967 


(0.06598726114649682, 0.013261378828886967)

In [12]:
compressed_evaluation.evaluate('./Models/resnet_50_f16_quant.tflite')

100%|██████████| 3925/3925 [03:45<00:00, 17.37it/s]

Accuracy of the model on 3925 images is 0.8300636942675159
Average prediction time per image was 0.05681291148920727 


(0.8300636942675159, 0.05681291148920727)

In [13]:
compressed_evaluation.evaluate('./Models/vgg16_f16_quant.tflite')

100%|██████████| 3925/3925 [10:00<00:00,  6.54it/s]

Accuracy of the model on 3925 images is 0.7579617834394905
Average prediction time per image was 0.15209060261963278 


(0.7579617834394905, 0.15209060261963278)

Evaluating Int8 Quantized Model

In [14]:
compressed_evaluation.evaluate('./Models/mobilenet_v2_int8_quant.tflite')

100%|██████████| 3925/3925 [00:58<00:00, 66.96it/s]

Accuracy of the model on 3925 images is 0.03159235668789809
Average prediction time per image was 0.014842959847419885 


(0.03159235668789809, 0.014842959847419885)

In [15]:
compressed_evaluation.evaluate('./Models/resnet_50_int8_quant.tflite')

100%|██████████| 3925/3925 [05:12<00:00, 12.55it/s]

Accuracy of the model on 3925 images is 0.4807643312101911
Average prediction time per image was 0.0793717786460925 


(0.4807643312101911, 0.0793717786460925)

In [16]:
compressed_evaluation.evaluate('./Models/vgg16_int8_quant.tflite')

100%|██████████| 3925/3925 [18:07<00:00,  3.61it/s]

Accuracy of the model on 3925 images is 0.39872611464968155
Average prediction time per image was 0.27645560252438683 


(0.39872611464968155, 0.27645560252438683)